### Data Processing

In [1]:
from tensorflow.examples.tutorials import mnist
mnist_data = mnist.input_data.read_data_sets("/tmp/data/")

/Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W0817 21:32:33.785245 4525913536 deprecation.py:323] From <ipython-input-1-f648f33866b9>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0817 21:32:33.786481 4525913536 deprecation.py:323] From /Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is 

Extracting /tmp/data/train-images-idx3-ubyte.gz


W0817 21:32:34.265812 4525913536 deprecation.py:323] From /Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0817 21:32:34.389964 4525913536 deprecation.py:323] From /Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [104]:
import numpy as np


train_indxs_upto_4 = mnist_data.train.labels<5
train_images_upto_4 = mnist_data.train.images[train_indxs_upto_4]
train_labels_upto_4 = mnist_data.train.labels[train_indxs_upto_4]

val_indxs_upto_4 = mnist_data.validation.labels<5
val_images_upto_4 = mnist_data.validation.images[val_indxs_upto_4]
val_labels_upto_4 = mnist_data.validation.labels[val_indxs_upto_4]

test_indxs_upto_4 = mnist_data.test.labels<5
test_images_upto_4 = mnist_data.test.images[test_indxs_upto_4]
test_labels_upto_4 = mnist_data.test.labels[test_indxs_upto_4]


### Function Definitions 

In [120]:
import tensorflow as tf
from tensorflow.contrib.layers import variance_scaling_initializer as he_initializer
from tensorflow.nn import sparse_softmax_cross_entropy_with_logits as softmax_xentropy
from tensorflow.layers import dense
import numpy as np


def get_leaky_relu(alpha):
    return lambda z, name=None: tf.maximum(alpha*z,z, name=name)

def get_logits(x, layer_sizes):
    
    activation = get_leaky_relu(alpha=0.3)
#     activation = tf.nn.elu
    initializer = he_initializer()
    
    with tf.name_scope("DNN"):
        layer_input = x
        for i, layer_size in enumerate(layer_sizes[2:]):
            layer_name = "input" if i==0 else \
                ("output" if i==len(layer_sizes)-1 else "hidden%d"%i)
            
            act = None if i==len(layer_sizes)-1 else activation
            
            layer_input = dense(layer_input, layer_size, name=layer_name, 
                            kernel_initializer=initializer, activation= act)
            
    return layer_input


def get_softmax_xentropy_loss(logits,y):
    with tf.name_scope("loss"):
        xentropy = softmax_xentropy(labels=y, logits=logits)
        return tf.reduce_mean(xentropy)

def get_optimizer(loss, learning_rate=0.01):
    with tf.name_scope("train"):
        optimizer =  tf.train.AdamOptimizer (learning_rate=learning_rate)
        optimizer_op = optimizer.minimize(loss)
    return optimizer_op

def get_validation_score(logits,y):
    with tf.name_scope("validation"):
        preds = tf.nn.in_top_k(logits,y,1)
        return tf.reduce_mean(tf.cast(preds, dtype=np.float32))
    
def get_batch(x,y,batch_size):
    n_batches = len(y)//batch_size + 1
    for i in range(n_batches):
        indxes = np.random.choice(len(y), size=batch_size, replace=False)
        yield x[indxes], y[indxes]


### Training

In [133]:
from my_libs.tf_checkpoint import CheckpointSaver
from my_libs.tf_graph_saver import ScalerGraphSaver

def train_model(train_ds, validation_ds, restore_checkpoint=True):
    layer_sizes = [100,100,100,100,100]
    learning_rate = 0.01
    n_epochs = 200
    batch_size = 50

    train_x, train_y = train_ds
    val_x, val_y = validation_ds
    
    tf.reset_default_graph()
    
    var_epoch = tf.Variable(initial_value=0,dtype=np.int16, name="epoch")
    inc_epoch = tf.compat.v1.assign_add(var_epoch,1, name="inc_epoch")
    
    x = tf.placeholder(shape=(None, 28*28), dtype=np.float32,name="x")
    y = tf.placeholder(shape=(None), dtype=np.int32,name="y")
    logits = get_logits(x, layer_sizes)
    
    loss_op = get_softmax_xentropy_loss(logits,y)
    optimizer = get_optimizer(loss_op)
    validation_score = get_validation_score(logits,y)
    
    with tf.Session() as sess:
        chk_saver = CheckpointSaver(name="excercise1_epoch")
        
        epoch_start=0
        if restore_checkpoint and chk_saver.restore_checkpoint():
            epoch_start = var_epoch.eval(sess)
            print("restored at epoch %d"%epoch_start)
        else:
            init = tf.global_variables_initializer()
#             init = tf.variables_initializer([var_epoch])
            sess.run(init)
        
        
        
        for epoch in range(epoch_start, n_epochs):
            for batch_x, batch_y in get_batch(train_x,train_y, batch_size):
                loss,_ = sess.run([loss_op, optimizer], feed_dict={x:batch_x,y:batch_y})
            sess.run(inc_epoch)
            if (epoch>0 and (epoch%25==0 or epoch==n_epochs-1)):
                chk_saver.save_checkpoint()
                print("%d: train-%s, validation-%s,"% \
                      (epoch, loss, sess.run(loss_op, feed_dict={x:val_x,y:val_y})))
                print(sess.run(validation_score, feed_dict={x:val_x,y:val_y}))
    
        
        
        
        
        
    def predict(test_x):
        return tf.math.argmax(logits, axis=1)
    return predict
    

In [135]:
predict = train_model((train_images_upto_4,train_labels_upto_4),
                      (val_images_upto_4,val_labels_upto_4), restore_checkpoint=True)

predictions = predict(test_images_upto_4)
accuracy = np.mean(test_labels_upto_4==predictions)

restored at epoch 150
150: train-0.0, validation-6.30893,
0.988663
175: train-0.0, validation-4.5375624,
0.9882721
199: train-1.8938293, validation-6.8174853,
0.98944485


In [113]:
chk_saver = CheckpointSaver(name="excercise1_epoch")

In [6]:
dense?

In [69]:
tf.train.AdamOptimizer?

In [76]:
train_labels.reshape(-1,1).shape

(55000, 1)

In [99]:
tf.nn.softmax_cross_entropy_with_logits_v2?

In [86]:
np.random.choice?